In [1]:
import sys
sys.path.insert(1, '../code')

In [2]:
import os
import random
import itertools
import numpy as np 
import pandas as pd 

from tqdm import tqdm

from utils import * 
from ranker import BM25, Ranker
from relevance import run_relevance_tests


<h2>Loading & indexing</h2>

In [3]:
docs = load_docs("../data/ask_reddit_posts_augmented.jsonl")

<h2>Load queries</h2>

In [4]:
SPLIT_SIZE = 0.6

In [5]:
queries_to_relevance = load_true_relevance("../eval/reformated_relevance_scores.csv")
queries = list(queries_to_relevance.keys())

In [6]:
N_TUNE_QUERIES = int(np.round(len(queries) * SPLIT_SIZE))

random.shuffle(queries)

tune_queries = queries[:N_TUNE_QUERIES]
val_queries = queries[N_TUNE_QUERIES:]

<h2>Param Grid</h2>

<a href = "https://www.elastic.co/blog/practical-bm25-part-3-considerations-for-picking-b-and-k1-in-elasticsearch">reference</a>

In [7]:
PARAM_GRID = {'b': [0.05, 0.1, 0.2, 0.25, 0.3, 0.35, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1], 
              'k1': [0, 0.25, 0.5, 1, 1.25, 1.5, 2, 2.25, 2.5, 3, 3.25, 3.5, 4, 4.5, 5], 
              'k3': [0, 0.25, 0.5, 1, 1.25, 1.5, 2, 2.25, 2.5, 3, 3.25, 3.5, 4, 4.5, 5]}

In [8]:
params, vals = zip(*PARAM_GRID.items())
param_permutations = [dict(zip(params, v)) for v in itertools.product(*vals)]

<h2>Load IR Infastructure</h2>

In [9]:
scores = {}

for permutation in tqdm(param_permutations):
    b = permutation['b']
    k1 = permutation['k1']
    k3 = permutation['k3']

    #eval = run_relevance_tests(tune_queries, '', ranker, 10)
    avg_map = eval['map']
    avg_ncdg = eval['ncdg']

    scores[(b, k1, k3)] = (avg_map, avg_ncdg)

NameError: name 'cutoff' is not defined

In [ ]:
sorted_scores = sorted(list(scores.items()), key = lambda s: s[1][1], reverse = True)